# KraaiS-pairs

KraaiS-Pairs is a challenge dataset to measuring the degree of undersirable bias is present in Language models. This version of the 'CrowS-pairs' model uses Dutch sentence pairs in order to detect bias in Dutch language models. The code used originates from 'CrowS-Pairs: A Challenge Dataset for Measuring Social Biases in Masked Language Models' paper(Nikita, et al.2020).

For this research some adjustments have been made to the code. It is now possible to specify which bias types are taken into account. Also the main function has been altered so that a context can be added to the sentence pairs.

In [1]:
import os
import csv
import json
import math
import torch
import random
import argparse
import difflib
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from collections import defaultdict
from tqdm import tqdm
from scipy import stats

To create a small challenge set, choose 13 random sentences

In [2]:
randomlist = random.sample(range(91), 13)
print(randomlist)

[82, 21, 30, 77, 0, 49, 64, 18, 9, 11, 15, 23, 40]


In [3]:
def read_data(input_file, bias, randomlist, sentence_type, context_type):
    """
    Load data into pandas DataFrame format.
    """
    if isinstance(bias, list) == False:
        print('Bias type needs to be a list!')
        return
    
    
    index = j = 0
    
    df_data = testdata_df = context_df = pd.DataFrame(columns=['index', 'sent1', 'sent2', 'direction', 'bias_type', 'anti_stereo'])
    
    with open(input_file, mode="r", encoding="latin1") as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            direction, gold_bias = '_', '_'
            direction = row['stereo_antistereo']
            bias_type = row['bias_type']
            anti_stereo = row['stereo_antistereo']


            sent1, sent2 = '', ''
            if direction == 'stereo':
                sent1 = row['sent_more']
                sent2 = row['sent_less']
            else:
                sent1 = row['sent_less']
                sent2 = row['sent_more']

            df_item = {'index': index,
                       'sent1': sent1,
                       'sent2': sent2,
                       'direction': direction,
                       'bias_type': bias_type,
                       'anti_stereo' : anti_stereo}
            
            
            # create a context dataframe and a challenge set
            if sentence_type == context_type:
                for i in bias:
                    if i == bias_type and anti_stereo == sentence_type:
                        df_data = df_data.append(df_item, ignore_index=True)
                        index+=1
                        if index in randomlist:
                            testdata_df = testdata_df.append(df_item, ignore_index=True)
                        elif j < 2 and anti_stereo == context_type:
                            context_df = context_df.append(df_item, ignore_index=True)
                            j += 1
            if sentence_type != context_type:
                for i in bias:
                    if i == bias_type and anti_stereo == sentence_type:
                        df_data = df_data.append(df_item, ignore_index=True)
                        index+=1
                        if index in randomlist:
                            testdata_df = testdata_df.append(df_item, ignore_index=True)
                    elif i == bias_type and anti_stereo == context_type and j < 2:
                        context_df = context_df.append(df_item, ignore_index=True)
                        j += 1
                                                       


    return testdata_df, context_df

# Function

In [6]:
def get_log_prob_unigram(masked_token_ids, token_ids, mask_idx, lm):
    """
    Given a sequence of token ids, with one masked token, return the log probability of the masked token.
    """
    
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]
    
    # get model hidden states
    output = model(masked_token_ids)
    hidden_states = output[0].squeeze(0)
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)

    # we only need log_prob for the MASK tokens
    assert masked_token_ids[0][mask_idx] == mask_id

    hs = hidden_states[mask_idx]
    target_id = token_ids[0][mask_idx]
    log_probs = log_softmax(hs)[target_id]

    return log_probs

In [7]:
def get_span(seq1, seq2):
    """
    This function extract spans that are shared between two sequences.
    """

    seq1 = [str(x) for x in seq1.tolist()]
    seq2 = [str(x) for x in seq2.tolist()]

    matcher = difflib.SequenceMatcher(None, seq1, seq2)
    template1, template2 = [], []
    for op in matcher.get_opcodes():
        # each op is a list of tuple: 
        # (operation, pro_idx_start, pro_idx_end, anti_idx_start, anti_idx_end)
        # possible operation: replace, insert, equal
        # https://docs.python.org/3/library/difflib.html
        if op[0] == 'equal':
            template1 += [x for x in range(op[1], op[2], 1)]
            template2 += [x for x in range(op[3], op[4], 1)]

    return template1, template2

In [8]:
def mask_unigram(data, lm, context, n=1):
    """
    Score each sentence by masking one word at a time.
    The score for a sentence is the sum of log probability of each word in
    the sentence.
    n = n-gram of token that is masked, if n > 1, we mask tokens with overlapping
    n-grams.
    """
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]

    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')

    sent1, sent2 = data["sent1"], data["sent2"]
    

    if uncased:
        sent1 = sent1.lower()
        sent2 = sent2.lower()
        
    # word length
    words = context.split()
    totalwords = len(words)

    # tokenize
    sent1_token_ids = tokenizer.encode(context+sent1, return_tensors='pt')
    sent2_token_ids = tokenizer.encode(context+sent2, return_tensors='pt')
    
    # get spans of non-changing tokens
    template1, template2 = get_span(sent1_token_ids[0], sent2_token_ids[0])

    assert len(template1) == len(template2)

    N = len(template1)  # num. of tokens that can be masked
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)
    
    sent1_log_probs = 0.
    sent2_log_probs = 0.
    total_masked_tokens = 0

    # skipping CLS and SEP tokens, they'll never be masked
    for i in range(totalwords, N-1):
        sent1_masked_token_ids = sent1_token_ids.clone().detach()
        sent2_masked_token_ids = sent2_token_ids.clone().detach()

        sent1_masked_token_ids[0][template1[i]] = mask_id
        sent2_masked_token_ids[0][template2[i]] = mask_id
        total_masked_tokens += 1

        score1 = get_log_prob_unigram(sent1_masked_token_ids, sent1_token_ids, template1[i], lm)
        score2 = get_log_prob_unigram(sent2_masked_token_ids, sent2_token_ids, template2[i], lm)

        sent1_log_probs += score1.item()
        sent2_log_probs += score2.item()

    score = {}
    # average over iterations
    score["sent1_score"] = sent1_log_probs
    score["sent2_score"] = sent2_log_probs

    return score

In [9]:
def evaluate(input_file, lm_model, bias_type, test_list, sentence_type, context_type):
    """
    Evaluate a masked language model using CrowS-Pairs dataset.
    """

    if isinstance(bias_type, list) == False:
        print('bias type needs to be a list!')
        return
    
    print("Evaluating:")
    print("Input:", input_file)
    print("Model:", lm_model)
    print("=" * 100)

    logging.basicConfig(level=logging.INFO)

    # load data into panda DataFrame
    df_data, context_df = read_data(input_file, bias_type, test_list, sentence_type, context_type)
    
    # Make context dataframe into a string
    context_string = ''

    for k in context_df['sent1']:
        context_string = context_string + ' ' + k
        
    if lm_model == "bert":
                tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                model = BertForMaskedLM.from_pretrained('bert-base-uncased')
                uncased = True
                
    elif lm_model == "roberta":
        tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
        model = RobertaForMaskedLM.from_pretrained('roberta-large')
        uncased = False

    elif lm_model == "robbert":
        tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
        model = RobertaForMaskedLM.from_pretrained("pdelobelle/robbert-v2-dutch-base")
        uncased = True

    elif lm_model == "bertje":
        tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
        model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
        uncased = True


    model.eval()
    if torch.cuda.is_available():
        model.to('cuda')


    mask_token = tokenizer.mask_token
    log_softmax = torch.nn.LogSoftmax(dim=0)
    vocab = tokenizer.get_vocab()
    with open(lm_model + ".vocab", "w") as f:
        f.write(json.dumps(vocab))

    lm = {"model": model,
          "tokenizer": tokenizer,
          "mask_token": mask_token,
          "log_softmax": log_softmax,
          "uncased": uncased
    }

    # score each sentence. 
    # each row in the dataframe has the sentid and score for pro and anti stereo.
    df_score = pd.DataFrame(columns=['sent_more', 'sent_less', 
                                     'sent_more_score', 'sent_less_score',
                                     'score', 'stereo_antistereo', 'bias_type'])


    total_stereo, total_antistereo = 0, 0
    stereo_score, antistereo_score = 0, 0

    N = 0
    neutral = 0
    total = len(df_data.index)
    with tqdm(total=total) as pbar:
        for index, data in df_data.iterrows():
            
            direction = data['direction']
            bias = data['bias_type']
            score = mask_unigram(data, lm, context_string)

            for stype in score.keys():
                score[stype] = round(score[stype], 3)

            N += 1
            pair_score = 0
            pbar.update(1)
            if score['sent1_score'] == score['sent2_score']:
                neutral += 1
            else:
                if direction == 'stereo':
                    total_stereo += 1
                    if score['sent1_score'] > score['sent2_score']:
                        stereo_score += 1
                        pair_score = 1
                elif direction == 'antistereo':
                    total_antistereo += 1
                    if score['sent2_score'] > score['sent1_score']:
                        antistereo_score += 1
                        pair_score = 1

            sent_more, sent_less = '', ''
            if direction == 'stereo':
                sent_more = data['sent1']
                sent_less = data['sent2']
                sent_more_score = score['sent1_score']
                sent_less_score = score['sent2_score']
            else:
                sent_more = data['sent2']
                sent_less = data['sent1']
                sent_more_score = score['sent1_score']
                sent_less_score = score['sent2_score']

            df_score = df_score.append({'sent_more': sent_more,
                                        'sent_less': sent_less,
                                        'sent_more_score': sent_more_score,
                                        'sent_less_score': sent_less_score,
                                        'score': pair_score,
                                        'stereo_antistereo': direction,
                                        'bias_type': bias
                                      }, ignore_index=True)

    print(df_score)

    print('=' * 100)
    print('Total examples:', N)
    print('Metric score:', round((stereo_score + antistereo_score) / N * 100, 2))
    print('=' * 100)
    print()

def evaluate(input_file, lm_model, bias_type, test_list, sentence_type, context_type):

In [11]:
evaluate('data/crows_pairs_anonymized.csv', 'bert', ['gender'],  randomlist, 'stereo', 'stereo')